<a href="https://colab.research.google.com/github/coleaderme/multi-yt/blob/main/multiYT_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Default: Max video length 40 mins. (See Download menu)
        

No limit on Audio.
         

In [ ]:
LINK = "" # @param {type:"string"}
Download = "Audio_Video" #@param ["Audio_Only", "Audio_Video", "Bypass_40Mins_Video_Limit"]

from google.colab import drive
drive.mount('/content/drive')

!wget "https://raw.githubusercontent.com/coleaderme/multi-yt/main/multiYT.sh"
!wget "https://raw.githubusercontent.com/coleaderme/multi-yt/main/multiYT_A.sh"
!wget -c "https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp_linux" && mv yt-dlp_linux /usr/bin/yt-dlp && chmod +x /usr/bin/yt-dlp
## Not using aria2c, already downloading via 10 parallel connections.
# !apt -qq update && apt -qq install aria2

#### Step 1: Command to Flat URL to playlist (videos less then 40 mins)
if Download == "Audio_Video":
    !yt-dlp --match-filter "duration < 2400" --flat-playlist -i --print-to-file url playlist.txt "$LINK"
elif Download == "Audio_Only":
    !yt-dlp --flat-playlist -i --print-to-file url playlist.txt "$LINK"
elif Download == "Bypass_40Mins_Video_Limit":
    !yt-dlp --flat-playlist -i --print-to-file url playlist.txt "$LINK"
else:
    print("Expected 1 of 3 opts above.")

## download X numbers of videos at once.
INCREMENT = 10


#### Step 2: Convert large list into smaller list of n items
with open("playlist.txt") as pl:
    links = [l.strip() for l in pl]

total = len(links)
groups = int(total / INCREMENT)
extra = total % INCREMENT

batch = 0
nextj = 0 # pass on the local variable, i.e, for loop.
## loop 11 times of 5's
for j in range(groups):
    ## [quick] how does this works? ..
    ## links[0:0+5] >> links[5:5+5] >> links[10:10+5]
    with open(f"queue{j+1}.txt", 'a') as f:
        for k in links[batch:batch+INCREMENT]:
            f.write(f"{k}\n")
        batch = batch + INCREMENT
    nextj = j # pass on the local variable to next loop.

if extra:
    with open(f"queue{nextj+1+1}.txt", 'a') as f:
        for r in links[batch:]:
            f.write(f"{r}\n")
            # print("extra "+ r)

print("total items:", total,"groups:", groups ,"extra:", extra)

#### Step 3: Execute this command
if Download == "Audio_Only":
    !for i in $(ls | grep "que"); do bash multiYT_A.sh $i;done
else:
    !for i in $(ls | grep "que"); do bash multiYT.sh $i;done

#*******************************************************************************************
#### Step 4: Packing and moving
!mkdir YT_Downloads
!mv *.mkv YT_Downloads/
!mv *.webm YT_Downloads/
!mv *.opus YT_Downloads/

!7z a -m0=copy YT_Downloads.7z "/content/YT_Downloads/"

print('Moving to Google Drive...')
!cp -v "/content/YT_Downloads.7z" "/content/drive/MyDrive/"
print('[OK] Done.')